In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("/Users/ashabanov/code/metric_learning/open-metric-learning")

from IPython.core.display import HTML
from IPython.display import display
import pandas as pd

display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_rows', 330)

import numpy as np
from functools import lru_cache
from collections import defaultdict, Counter
from pathlib import Path
import pytorch_lightning as pl
import torch
import torchvision
import random
from sklearn.model_selection import train_test_split

from oml.functional.metrics import calc_retrieval_metrics
from oml.utils.misc_torch import pairwise_dist
from oml.functional.metrics import calc_gt_mask, calc_mask_to_ignore
    
from oml.datasets.base import DatasetQueryGallery, DatasetWithLabels
from oml.lightning.modules.retrieval import RetrievalModule
from oml.lightning.callbacks.metric import MetricValCallback
from oml.losses.triplet import TripletLossWithMiner
from oml.metrics.embeddings import EmbeddingMetrics
from oml.miners.inbatch_all_tri import AllTripletsMiner
from oml.miners.inbatch_nhard_tri import NHardTripletsMiner
from oml.models.vit.vit import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.utils.download_mock_dataset import download_mock_dataset
from oml.interfaces.models import IExtractor

from source import get_datasets, ProjExtractor, PrecisionCriterion

%matplotlib inline


In [ ]:
# todo: check convergency after using dino features on inshop
# todo: check the loss calculation


In [ ]:
# train_dataset, val_dataset = get_datasets(
#     Path("/nydl/data/DeepFashion_InShop"), "df_no_bboxes.csv", "embeddings_vits16_dino.pkl"
# )

train_dataset, val_dataset = get_datasets(
    Path("/nydl/data/Stanford_Online_Products"), "df.csv", "embeddings_vits16_sop.pkl"
)

In [ ]:
model = ProjExtractor(normalise_features=True)

# train
optimizer = torch.optim.SGD(model.parameters(), lr=3) # todo
# criterion = TripletLossWithMiner(margin=0.15, miner=NHardTripletsMiner(n_positive=1, n_negative=1))
criterion = PrecisionCriterion()
batch_sampler = BalanceSampler(train_dataset.get_labels(), n_labels=800, n_instances=3)  # 3985, 11318
train_loader = torch.utils.data.DataLoader(train_dataset, batch_sampler=batch_sampler, num_workers=10)

# val
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=10000, shuffle=False, num_workers=10)
metric_callback = MetricValCallback(metric=EmbeddingMetrics(cmc_top_k=(1,5)))

# run
pl_model = RetrievalModule(model, criterion, optimizer)
trainer = pl.Trainer(max_epochs=1000, check_val_every_n_epoch=1, callbacks=[metric_callback], num_sanity_val_steps=0, gpus=1, enable_checkpointing=False)
trainer.fit(pl_model, train_dataloaders=train_loader, val_dataloaders=val_loader)
# trainer.validate(model=pl_model, val_dataloaders=val_loader)


In [ ]:
# batch = next(iter(train_loader))
# emb = batch["input_tensors"]
# labels = batch["labels"]

# emb.requires_grad = True

# criterion = PrecisionCriterion()
# loss = criterion(emb, labels)
# loss.backward()

# print(loss)


In [ ]:
# from source import precision_loss

# emb = val_dataset.embeddings
# emb.requires_grad = True

# labels = torch.tensor(val_dataset.df["label"].to_list())

# is_query   = torch.tensor(val_dataset.df["is_query"].to_list()).bool()
# is_gallery = torch.tensor(val_dataset.df["is_gallery"].to_list()).bool()

# loss = precision_loss(emb, is_query, is_gallery, labels)

# loss.backward()


In [ ]:
from source import PrecisionCriterion

mask_gt = torch.tensor([[1, 0, 0, 1], [1, 1, 0, 1]], requires_grad=False).bool()
distances = torch.tensor([[10.1, 3.2, 0.3, 2.4], [0.1, 3.2, 5.3, 2.4]], requires_grad=True)

k = 2

emb = torch.randn((200,64), requires_grad=True)
labels = torch.ones(200)
labels[100:] = 2

p1 = PrecisionCriterion()(emb, labels)
p2 = calc_retrieval_metrics(distances, mask_gt=mask_gt, precision_top_k=(k,))["precision"][k]

print(p1, p2)

p1.backward()
